# Unit Tests

The following is a random collection of Asteroid programs each testing a particular aspect of Asteroid.  All the programs in this notebook should execute.  

In [1]:
from asteroid_interp import interp

In [4]:
program =\
'''
load "util.ast".
load "io.ast".

let cnt = integer(input("Please enter an integer value: ")).

for i in 1 to cnt do
    print i.
end for
'''

interp(program)

Please enter an integer value:  5
1 
2 
3 
4 
5 


In [ ]:
program =\
'''
load "standard.ast".
load "io.ast".

let name = input("Please enter your name: ").
print("Hello" + name + "!").
'''

interp(program)

In [ ]:
program = \
'''
load "util.ast".
load "io.ast".

constructor S with arity 1.

let x = 'S(S(0)).
let y = 'S(S(x)).
let z = y.

print y.
print z.
print (eval (z)).
'''
interp(program, tree_dump=False, symtab_dump=False, do_walk=True, exceptions=True)

In [ ]:
program = \
'''
constructor S with arity 1.

let y = S S  S S 0.

'''
interp(program, tree_dump=True, symtab_dump=True, do_walk=True, exceptions=True)

In [ ]:
program = \
'''
function ident 
    with n do 
        return n 
    end function 

let y = ident ident  0.

'''
interp(program, tree_dump=True, symtab_dump=True, do_walk=True, exceptions=True)

In [ ]:
program = \
'''
function ident 
    with n do 
        return n 
    end function 

let y = ident(ident (0)).
let x = ident ident 0.

'''
interp(program, tree_dump=True, symtab_dump=True, do_walk=True, exceptions=True)

In [ ]:
program = \
'''
-- Factorial

load "standard.ast".
load "io.ast".

function fact 
    with 0 do
        return 1
    orwith n do
        return n * fact (n-1).
    end function

print ("The factorial of 3 is: " + fact (3)).
'''

interp(program, exceptions=False, symtab_dump=False)

In [ ]:
program = \
'''
-- show that the value constructed by head-tail is a list
let [1,2,3] = 1 | [2,3].

-- show that a list can be decomposed with head-tail
let 1 | [2,3] = [1,2,3].

-- show that we can nest head-tail operators
let [1,2,3] = 1 | 2 | 3 | [].

'''
interp(program, tree_dump=True, symtab_dump=False, do_walk=True, exceptions=False)

In [ ]:
program = \
'''
load "standard.ast".
load "util.ast".
load "io.ast".

try

    throw Error "--- error ---".
    
catch Error(msg) do
    print msg.

end try

'''
interp(program, tree_dump=False, symtab_dump=False, do_walk=True, exceptions=False)

In [ ]:
program = \
'''
load "standard.ast".
load "io.ast".

let h|t = [1,2,3].
print ("head: " + h + " tail: " + t).
'''
interp(program, tree_dump=False, symtab_dump=False, do_walk=True, exceptions=False)

In [ ]:
program = \
'''
load "standard.ast".
load "io.ast".

let y = -1.
let x = 4 if y == 3 else 0.
print x.

'''
interp(program, tree_dump=False, symtab_dump=False, do_walk=True, exceptions=False)

In [ ]:
program = \
'''
load "io.ast".

let y = none.
let x = y otherwise 1.
print x.

'''
interp(program, tree_dump=False, symtab_dump=False, do_walk=True, exceptions=False)

In [ ]:
program = \
'''
load "standard.ast".
load "io.ast".

for x in 0 to 10 do
    print x.
    if x == 5 do
        break.
    end if
end for
'''
interp(program, tree_dump=False, do_walk=True, exceptions=False)

In [ ]:
program = \
'''
load "io.ast".

with a=1, b=2 do
    print(a,b).
end with
'''
interp(program, tree_dump=False, do_walk=True, symtab_dump=False, exceptions=True)

In [ ]:
program = \
'''
load "io.ast".

for (x,y) in [(1,1), (2,2), (3,3)]  do
    print (x,y).
end for

-- use unification as a filter
for (2,y) in [(1,11), (1,12), (1,13), (2,21), (2,22), (2,23)]  do
    print y.
end for
'''
interp(program, tree_dump=False, do_walk=True, symtab_dump=False)

In [ ]:
program = \
'''
load "io.ast".

for x in 1 to 10 do
    print x.
end for
'''
interp(program, tree_dump=False, do_walk=True, symtab_dump=False)

In [ ]:
program = \
'''
load "standard.ast".
load "io.ast".

let x = 42.

if x < 0 do
    let x = 0.
    print("Negative changed to zero").

elif x == 0 do
    print("Zero").

elif x == 1 do
    print("Single").

else do
    print("More").
    
end if

'''
interp(program, tree_dump=False, do_walk=True, exceptions=False)

In [ ]:
program = \
'''
load "standard.ast".
load "io.ast".

let x = 1.
while x <= 10 do
    print x.
    let x = x + 1.
end while
'''
interp(program, exceptions=True)

In [ ]:
program = \
'''
let 1 = 1.
'''
interp(program, exceptions=True, tree_dump=True)

In [ ]:
program = \
'''
load "standard.ast".

'''
interp(program, tree_dump=False, symtab_dump=False)

In [ ]:
program = \
'''
load "io.ast".

constructor A with arity 2.

let a = A("Hello", (lambda with self do return self@[0])).
print (a@[1] a).
'''
interp(program, tree_dump=False, symtab_dump=False, exceptions=False)

In [ ]:
program = \
'''
load "io.ast".

constructor A with arity 2.

let a = A("Hello", (lambda with self do print self@[0])).
a@[1] a.
'''
interp(program, tree_dump=False, symtab_dump=False, exceptions=False)

In [ ]:
program = \
'''
load "io.ast".

print (1,2,3).
'''
interp(program)

In [ ]:
program = \
'''
let nl = [[1 to 10],].
'''
interp(program, tree_dump=True, symtab_dump=True)

In [ ]:
program = \
'''
-- let l = [2*i where i in [1 to 100]].
'''
interp(program, symtab_dump=True, tree_dump=True, exceptions=False)

In [ ]:
program = \
'''
let [0 to 10] = '[0 to 10].

let '[0 to 10] = '[0 to 10].

'''
interp(program, symtab_dump=False, tree_dump=False, exceptions=True)

In [ ]:
program = \
'''
constructor A with arity 3.

let a = A(1,2,3).
let b = a@[0 to 2].
'''
interp(program, symtab_dump=True, tree_dump=True, exceptions=True)

In [ ]:
program = \
'''
let a = [0,1,2,3].
let b = a@[0 to 3 step 2].
'''
interp(program, symtab_dump=True, tree_dump=True, exceptions=True)

In [ ]:
program = \
'''
let x = 1.
let y = [0 to 10 step 2].
let z = 4 in y.
'''
interp(program, symtab_dump=True, tree_dump=True)

In [ ]:
in_test = \
'''
let y = 3 in [1,2,3].
'''
interp(in_test, symtab_dump=True)

In [ ]:
is_test = \
'''
let y = (1,2) is (1,x).
let z = (1,3) is (1,x).

'''
interp(is_test, symtab_dump=True)

In [ ]:
constr_dict_store =\
'''
constructor A with arity 2.

let foo = A(("a",1),("b",2)).
let foo@{"a"} = 2.

'''
interp(constr_dict_store, tree_dump=True, exceptions=True, symtab_dump=True)

In [ ]:
store_dict = \
'''
let d = [("a", 100)].
let d@{"foo"} = 1.
let d@{"goo"} = 2.

let d@{"foo"} = 2.

'''
interp(store_dict, tree_dump=True, symtab_dump=True, exceptions=False)

In [ ]:
store_list = \
'''
load "io.ast".

let b = [[1,2,3],
         [4,5,6],
         [7,8,9]].
let b@[1]@[1] = 0.
print b.
'''
interp(store_list, tree_dump=True, exceptions=True)

In [ ]:
constr_dict =\
'''
load "io.ast".

constructor A with arity 2.

let foo = A(("a",1),("b",2)).
print (foo@{"a"}).

let goo = A([(1,1),(2,2)]).
print (goo@{1}).


'''
interp(constr_dict, tree_dump=True, exceptions=True)

In [ ]:
dict = \
'''
load "io.ast".

let d = [("a", 1)].

print (d@{"a"}).
'''
interp(dict, tree_dump=True)

In [ ]:
list_equiv = \
'''
-- some list equivalencies

let [0] = 0, .
let [0,1,2] = 0,1,2.
let [[0,1,2]] = ((0,1,2),).
let (0,1,2),(3,4,5) = ((0,1,2),(3,4,5)).
'''
interp(list_equiv, tree_dump=True)

In [ ]:
unary_ops = \
'''
let x = -1.
let y = not true.
'''
interp(unary_ops, tree_dump=True, symtab_dump=True)

In [ ]:
apply = \
'''
-- infix operators are just a shorthand for prefix terms 
-- using special names for the operators

--load "standard.ast".

let x = __plus__ (1,1).
let y = 1 + 1.

let 1 + 1 = '__plus__ (1,1).
'''
interp(apply, tree_dump=True, symtab_dump=True)

In [ ]:
constr_lval = \
'''
constructor A with arity 1.
let A(0,) = A[0].  -- BUT A(0) != A[0] --> needs to be straightened out...
let a = A[0].
let a@0 = 1.
'''
interp(constr_lval, tree_dump=True, symtab_dump=True, exceptions=False)

In [ ]:
constr_lval2 = \
'''
constructor A with arity 3.
let a = A[1,2,3].
let a@[1] = 0.
'''
interp(constr_lval2, tree_dump=True, symtab_dump=True, exceptions=False)

In [ ]:
array_lval = \
'''
load "io.ast".

let a = [1,2,3].
let a@[2],a@[1],a@[0] = a.
print a
'''
interp(array_lval, tree_dump=False, symtab_dump=False, do_walk=True, exceptions=False)

In [ ]:
struct = \
'''
constructor A with arity 1.
constructor B with arity 2.

let x = A(1).
let y = B(1,2).

let A(z) = x.
let B(v,w) = y.

let xx = x@0.
let yy = y@[0,1].

'''
interp(struct, tree_dump=False, symtab_dump=True, exceptions=True)

In [ ]:
arr = \
'''
load "io.ast".

let v = 'a@[3].
print v.
'''
interp(arr)

In [ ]:
func = \
'''
load "standard.ast".
load "io.ast".

function inc with n do return n+1 end function

let v = inc(inc(0)).
let q = 1 + 1 + 1.
print (v, q).
'''
interp(func, tree_dump=False, symtab_dump=False)

In [ ]:
constr = \
'''
-- constructors and pattern matching

-- load "standard.ast".
load "io.ast".

constructor S with arity 1.
attach (lambda with n do return 1+n) to S.

let v = 'S(S(0)).
print v.

let v = S(S(0)).
print v.

let S (S (v))  = 'S(S(0)).
print v.

let S (S (0))  = 'S(S(0)).


let [[v]] = [[0]].
print v.

'''
interp(constr, tree_dump=False, symtab_dump=False, exceptions=False)

In [ ]:
string_conc = \
'''
-- show off our overloaded '+' operator
load "standard.ast".
load "io.ast".

print (1 + 1).

let s1 = "hello".
let s2 = "world".
let s3 = s1 + " " + s2 +"!".
print s3.

let l1 = [1,2,3].
let l2 = [4,5,6].
let l3 = l1 + l2.
print l3
'''
interp(string_conc, tree_dump=False, symtab_dump=False, exceptions=False)

In [ ]:
array_rval = \
'''
load "io.ast".

-- reverse the list
let a = [1,2,3].
let a = a@[2,1,0].
print a.

-- access multidim array
let b = [[1,2,3],
         [4,5,6],
         [7,8,9]].
let e = b@[1]@[1].
print e.
'''
interp(array_rval, tree_dump=False, do_walk=True, symtab_dump=False, exceptions=False)

In [ ]:
graphics = \
'''
-- Asteroid graphics
-- we can incorporate Python graphics into Asteroid via
-- escaped Python code

function circle with x, y, r do escape
"
#########################################################
vx = float(state.symbol_table.lookup_sym('x')[1])
vy = float(state.symbol_table.lookup_sym('y')[1])
vr = float(state.symbol_table.lookup_sym('r')[1])

import matplotlib.pyplot as plt

circle = plt.Circle((vx, vy), vr, color='blue')
fig, ax = plt.subplots()
ax.add_artist(circle)
plt.show()
#########################################################
"
end function

-- call the escaped function
circle(.5, .5, .2)
'''
interp(graphics, exceptions=False, tree_dump=False, symtab_dump=False, do_walk=True)

In [ ]:
attach = \
'''
-- the 'add' function implements the behavior for 
-- the '+' operator for ints, reals, and strings.

load "io.ast".

function add with a, b do return escape 
"
###################################################################
# return register from 'escape' calls
global __retval__ 
from asteroid_support import promote

# 'promote' is the type promotion table for primitive
# builtin types which implement the type hierarchy:
#    integer < real < string
type = promote(state.symbol_table.lookup_sym('a')[0], 
               state.symbol_table.lookup_sym('b')[0])

# select the correct add according to type
if type == 'integer':
    __retval__ = ('integer', 
            int(state.symbol_table.lookup_sym('a')[1]) + 
            int(state.symbol_table.lookup_sym('b')[1]))

elif type == 'real':
    __retval__ = ('real', 
            float(state.symbol_table.lookup_sym('a')[1]) + 
            float(state.symbol_table.lookup_sym('b')[1]))

elif type == 'string':
    __retval__ = ('string', 
            str(state.symbol_table.lookup_sym('a')[1])+
            str(state.symbol_table.lookup_sym('b')[1]))

else:
    raise ValueError('unsupported type in add')
###################################################################
"     
end function

-- attach the behavior to the '+' constructor
attach add to __plus__.

-- test the '+' operator with different data types
let x = 1.2 + 2 .
print x.
print ("the output is: " + x).
-- print [1,2] + 3.
'''
interp(attach, tree_dump=False, symtab_dump=False, exceptions=False)

In [ ]:
interp('load "io.ast".let x = 1 + 2. print x.')

In [ ]:
interp('load "io.ast".let x = 1.3 . print x.')

In [ ]:
interp('load "io.ast".let 1,y = 1,2. print (1,y).')

In [ ]:
interp('load "io.ast".let x,2 = 1,2. print (x,2).')

In [ ]:
interp('load "io.ast".let s = 1,2. let x,y = s. print (x,y).')

In [ ]:
interp('load "io.ast".function ident with n do return n end function print (ident(2)).')

In [ ]:
interp('load "io.ast".let x = 1. function ident with n do return n end function print (ident( ident (x))) .', exceptions=True)

In [ ]:
interp("let _, x = [1], 2.", symtab_dump=True)

In [ ]:
lambda1 = \
'''
load "standard.ast"
load "io.ast".

print ((lambda with n do return n+1) 1).
'''
interp(lambda1, tree_dump=False, symtab_dump=False)